In [ ]:
try:
    %load_ext autotime
except:to
    !pip install ipython-autotime
    %load_ext autotime

IndentationError: ignored

In [ ]:
try:
    import deepface
except:
    !pip install deepface
    import deepface

In [ ]:
import numpy as np
import pandas as pd
import time
import requests
import os
import sys
import pickle
from lxml import html
import numpy as np
import urllib.request
import cv2
import matplotlib.pyplot as plt
from googlesearch import search
from deepface import DeepFace
from PIL import Image, ImageStat
import time
import logging

import warnings
warnings.filterwarnings("ignore")
tt = time.time
st = tt()

In [ ]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# for colab
colab = True
base_dir = "/content/drive/My Drive/nptel_face_recognition"
print(f"os.getcwd(): {os.getcwd()}")
if not os.getcwd() == base_dir:
    os.chdir(base_dir)
    print(f"os.getcwd(): {os.getcwd()}")

os.getcwd(): /content
os.getcwd(): /content/drive/My Drive/nptel_face_recognition


In [ ]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

def change_img_name(strn):
    temp = int(int(strn.split("/")[-1][:-4])/100)
    return temp

def stopPrint(func, *args, **kwargs):
    with open(os.devnull,"w") as devNull:
        original = sys.stdout
        sys.stdout = devNull
        res = func(*args, **kwargs)
        sys.stdout = original 
    return res

In [ ]:
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

ver = "_with_diversity"
known_faces_dir = os.path.join(data_dir, 'known_faces'+ver)
sgdf_known_faces_dir = os.path.join(data_dir, 'known_faces_sgdf')
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")
csv_files_dir = os.path.join(data_dir, "csv_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict"+ver+".pkl"))
prof_name_2_folder_name_dict = {v: k for k, v in folder_name_2_prof_name_dict.items()}

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('/')[-1])] for w in images_dir_list]


In [ ]:
face_recognition_results_df = pd.read_csv(os.path.join(csv_files_dir, "face_recognition_results"+ver+".csv"))
face_recognition_results_df.head()    

,Unnamed: 0,vname,nptel_channel_name,playlistn,vedio_path,true_name,accuracy,number of predictions,time,unique,counts,frame_index_list,names_list,slides_index_list_0.3,slides_index_list_0.4,slides_index_list_0.5,slides_index_list_0.6,slides_index_list_0.7,slides_index_list_0.8
0,0,ZDQBcuVRWYs.mp4,Aerospace Engineering,Jet and Rocket Propulsion,/content/drive/My Drive/nptel_face_recognition...,dr. a. kushari,0.90,50,23.459062,['dr. a. kushari' 'dr. p. gopinath ' 'prof. s....,[45 1 2 1 1],"[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...","['dr. a. kushari', 'dr. a. kushari', 'dr. a. k...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, ..."
1,1,QLcxx6MJnbA.mp4,Aerospace Engineering,Jet and Rocket Propulsion,/content/drive/My Drive/nptel_face_recognition...,dr. a. kushari,0.86,50,17.181391,['dr. a. kushari' 'prof. s. dasgupta' 'prof. s...,[43 2 2 3],"[16, 17, 18, 19, 20, 21, 40, 41, 42, 43, 44, 4...","['dr. a. kushari', 'dr. a. kushari', 'dr. a. k...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ..."
2,2,YrkbOeF7wFw.mp4,Aerospace Engineering,Jet and Rocket Propulsion,/content/drive/My Drive/nptel_face_recognition...,dr. a. kushari,0.30,50,33.226010,['dr. a. kushari' 'dr. p. gopinath ' 'prof. am...,[15 3 1 1 1 1 9 19],"[16, 17, 18, 19, 20, 21, 22, 23, 24, 42, 43, 4...","['dr. a. kushari', 'dr. a. kushari', 'prof. t....","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ...","[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 6, 7, 7, ..."
3,3,_9qwbichO1s.mp4,Aerospace Engineering,Introduction to Aerodynamics,/content/drive/My Drive/nptel_face_recognition...,prof. k. p. sinhamahapatra,0.98,50,13.712542,['prof. k. p. sinhamahapatra' 'prof. t.k.sengu...,[49 1],"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...","['prof. k. p. sinhamahapatra', 'prof. t.k.seng...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1...","[4, 5, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 12, 13, 1..."
4,4,BhWzgo7qkbE.mp4,Aerospace Engineering,Introduction to Aerodynamics,/content/drive/My Drive/nptel_face_recognition...,prof. k. p. sinhamahapatra,1.00,45,34.547672,['prof. k. p. sinhamahapatra'],[45],"[20, 21, 22, 23, 24, 25, 26, 85, 86, 98, 99, 1...","['prof. k. p. sinhamahapatra', 'prof. k. p. si...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,...","[5, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10,..."


In [ ]:
tracebacks_dir = os.path.join(data_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

face_recognition_results_df = pd.read_csv(os.path.join(csv_files_dir, "face_recognition_results"+ver+".csv"))

deep_face_metric = "cosine"
# True if min value is optimal
deep_face_metric_ascending = False
best_backend = "opencv"
temp_img_path = os.path.join(data_dir, "img.jpg")
num_models = 10
# serengil_deep_face_models_list = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "Ensemble"][:num_models]
serengil_deep_face_models_list = ["VGG-Face", "Facenet", "OpenFace"][:num_models]
# "DeepFace", "DeepID", "Dlib" taking too much time
# average time:
# "VGG-Face" - 2
# "Facenet" - 1
# "OpenFace" - 0
# "DeepFace" - 28
# "DeepID" > 20 mins

# start after labeled faces
# TODO: customize vedio start idx as all vedios dont take images from the first 2 vedios
# treshold for face verification
threshold = 0.25
max_pred_cnt = 2
vedio_ext = '.mp4'

num_vedios = 2

best_model_list = []
results_dict_list_dict = {model:[] for model in serengil_deep_face_models_list}
best_results_list_dict=[]
for index, row in face_recognition_results_df.iloc[:num_vedios].iterrows():
    vedio_path = row["vedio_path"]
    playlistn = row["playlistn"]
    nptel_channel_name = row["nptel_channel_name"]

    if playlistn not in playlist_2_prof_name_dict.keys():
        print(f"{playlistn} not in playlist_2_prof_name_dict")
        continue
            
    true_name = playlist_2_prof_name_dict[playlistn]
    if true_name is None:
        print(f"no name for {playlistn}")
        continue
        
    if true_name not in folder_name_2_prof_name_dict.values():
        print(f"no face for {true_name}")
        continue

    vname = vedio_path.split('/')[-1]

    video_capture = cv2.VideoCapture(vedio_path)

    vst = 20
    vet = 5*60
    num_frames_per_sec = 1

    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    st = tt()
    i=-1
    pred_cnt = 0
    y_pred_dict = {model:[] for model in serengil_deep_face_models_list}
    names_dict_list = {model:[] for model in serengil_deep_face_models_list}
    frame_index_dict_list = {model:[] for model in serengil_deep_face_models_list}
    while video_capture.isOpened():
        i+=1
        if int(i/fps) < vst:
            ret, frame = video_capture.read()
            continue

        if int(i/fps) > vet:
            break

          # Grab a single frame of video
        ret, frame = video_capture.read()

        # process
        if i%int(fps/num_frames_per_sec)==0:
            # TODO: PLS find a workaround this is too stupid
            cv2.imwrite(temp_img_path, frame)
            try:
                detected_face = DeepFace.detectFace(temp_img_path, detector_backend = best_backend)
                pred_cnt+=1
            except ValueError:
#                     except Exception as e:
#                         print(e)
                continue
            
            if pred_cnt>max_pred_cnt:
                break
                
            # for every model, for every labeled img folder, 
            # avearge distance from labeled and detected img
            for j, model in enumerate(serengil_deep_face_models_list):
                # TODO: find a way to not print warnings or tqdms
                print(f"model: {model}")
                # faster to pass build model for multiple calls
#                         actual_model = DeepFace.build_model(model)
                df = stopPrint(DeepFace.find, 
                                    img_path = temp_img_path, 
                                    detector_backend = best_backend,
#                                            model = actual_model,
                                    model_name = model,
                                    distance_metric = deep_face_metric,
                                    enforce_detection = False,
                                    db_path = sgdf_known_faces_dir)
                if df.empty:
                    y_pred_dict[model].append(0)
                    continue

                df['identity'] = df['identity'].apply(change_img_name)
                # distance_col = "distance"
                distance_col = model+"_"+deep_face_metric
                df = df.groupby('identity', as_index=False)[distance_col].mean()
                df = df.sort_values(distance_col, ascending=deep_face_metric_ascending)
                optim_folder_name = df.iloc[0]['identity']
                name = folder_name_2_prof_name_dict[optim_folder_name]
                names_dict_list[model].append(name)
                frame_index_dict_list[model].append(int(i/fps))
                if name==true_name:
                    y_pred_dict[model].append(1)
                else:
                    y_pred_dict[model].append(0)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()
    
    print(f"\n\nvedio: {vname}")
    print(f"true_name: {true_name}")
    y_pred_list_list = []
    for j, model in enumerate(serengil_deep_face_models_list):
        y_pred = y_pred_dict[model]
        y_pred_list_list.append(y_pred)
        names_list = names_dict_list[model]
        frame_index_list = frame_index_dict_list[model]
        print(f"\nmodel: {model}")
        print(f"unique predicted value counts:")
        unique, counts = np.unique(np.array(names_list), return_counts=True)
        print(np.asarray((unique, counts)).T)
        print(f"accuracy:{np.array(y_pred).mean()} \
        \nnumber of predictions: {len(y_pred)}")
        results_dict = {"vname": vname,
                    "nptel_channel_name": nptel_channel_name, 
                    "playlistn": playlistn,
                    "vedio_path": vedio_path,
                        "model": model,
                    "true_name": true_name,
                    "accuracy": np.array(y_pred).mean(),
                    "number of predictions": len(y_pred),
                    "time": tt()-st,
                    "unique": unique,
                    "counts": counts,
                    "frame_index_list":frame_index_list,
                    "names_list":names_list,
                    }
        results_dict_list_dict[model].append(results_dict)
        
    mean_y_pred_list = list(np.array(y_pred_list_list).mean(axis=1))
    best_model_idx = mean_y_pred_list.index(max(mean_y_pred_list))
    best_model = serengil_deep_face_models_list[best_model_idx]
    best_model_list.append(best_model)
    best_acc = mean_y_pred_list[best_model_idx]
    print(f"\nbest model: {model} \n accuracy:{best_acc} \n time: {tt()-st}")
    best_results_list_dict.append(results_dict_list_dict[best_model][-1])
    
    # save results csv files 
    for j, model in enumerate(serengil_deep_face_models_list):
        results_dict_list = results_dict_list_dict[model]
        results_dict_csv = pd.DataFrame(results_dict_list)
        # results_dict_csv.to_csv(os.path.join(csv_files_dir, f"serengil_deep_face_results_{model}.csv"))
    results_dict_csv = pd.DataFrame(best_results_list_dict)
    # results_dict_csv.to_csv(os.path.join(csv_files_dir, f"serengil_deep_face_results_best.csv"))

print("unique best model contender counts:")
unique, counts = np.unique(np.array(best_model_list), return_counts=True)
print(np.asarray((unique, counts)).T)
best_model = max(best_model_list, key=best_model_list.count)
print(f"\nbest of the best model: {best_model}")

# unique best model contender counts:
# [['Facenet' '21']
#  ['OpenFace' '10']
#  ['VGG-Face' '2']]

# best of the best model: Facenet
# time: 2h 46min 25s



Loading VGG-Face:   0%|          | 0/4 [00:00<?, ?it/s]       

model: Ensemble


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# tracebacks_dir = os.path.join(data_dir, "tracebacks")
# os.makedirs(tracebacks_dir, exist_ok = True)

# nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
# playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

# face_recognition_results_df = pd.read_csv(os.path.join(csv_dir, "face_recognition_results"+ver+".csv"))

# deep_face_metric = "cosine"
# # True if min value is optimal
# deep_face_metric_ascending = False
# best_backend = "opencv"
# temp_img_path = os.path.join(data_dir, "img.jpg")
# num_models = 10
# # serengil_deep_face_models_list = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "Ensemble"][:num_models]
# # "DeepFace"(28 secs), "DeepID", "Dlib" taking too much time

# # average time:
# # "VGG-Face" - 2
# # "Facenet" - 1
# # "OpenFace" - 0
# # "DeepFace" - 28
# # 
# serengil_deep_face_models_list = ["VGG-Face", "Facenet", "OpenFace"][:num_models]

# # start after labeled faces
# # TODO: customize vedio start idx as all vedios dont take images from the first 2 vedios
# vedio_start_idx = 2
# # treshold for face verification
# threshold = 0.25
# max_pred_cnt = 50
# vedio_ext = '.mp4'

# num_channels = 5
# num_playlists = 5
# num_vedios = 2
    
# # num_channels = 5
# # num_playlists = 5
# # num_vedios = 5

# best_model_list = []
# results_dict_list_dict = {model:[] for model in serengil_deep_face_models_list}
# best_results_list_dict=[]
# # for nptel_channel_name in nptel_channel_name_list:
# for nptel_channel_name in nptel_channel_name_list[1:num_channels+1]:
#     print(f'\n\n\n\nfor nptel_channel_name {nptel_channel_name}')
#     chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)

#     paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
#     paylist_name_list = [x.split('/')[-1] for x in paylist_path_list] 
    
#     # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
#     for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
#         print(f"\n\n\nplaylist: {playlistn}")
        
#         if playlistn not in playlist_2_prof_name_dict.keys():
#             print(f"{playlistn} not in playlist_2_prof_name_dict")
#             continue
            
#         true_name = playlist_2_prof_name_dict[playlistn]
#         if true_name is None:
#             print(f"no name for {playlistn}")
#             continue
            
#         if true_name not in folder_name_2_prof_name_dict.values():
#             print(f"no face for {true_name}")
#             continue
        
#         results_playlist_dir = os.path.join(results_chanel_dir, playlistn)
        
#         vedio_path_list = get_extension_files(playlist, ext='.mp4')
#         vedio_name_list = [x.split('/')[-1] for x in vedio_path_list] 
        
#         for vedio_path in vedio_path_list[vedio_start_idx:vedio_start_idx+num_vedios]:
#             vname = vedio_path.split('/')[-1]

#             video_capture = cv2.VideoCapture(vedio_path)

#             vst = 20
#             vet = 5*60
#             num_frames_per_sec = 1

#             fps = int(video_capture.get(cv2.CAP_PROP_FPS))
#             st = tt()
#             i=-1
#             pred_cnt = 0
#             y_pred_dict = {model:[] for model in serengil_deep_face_models_list}
#             names_dict_list = {model:[] for model in serengil_deep_face_models_list}
#             frame_index_dict_list = {model:[] for model in serengil_deep_face_models_list}
#             while video_capture.isOpened():
#                 i+=1
#                 if int(i/fps) < vst:
#                     ret, frame = video_capture.read()
#                     continue

#                 if int(i/fps) > vet:
#                     break

#                  # Grab a single frame of video
#                 ret, frame = video_capture.read()

#                 # process
#                 if i%int(fps/num_frames_per_sec)==0:
#                     # TODO: PLS find a workaround this is too stupid
#                     cv2.imwrite(temp_img_path, frame)
#                     try:
#                         detected_face = DeepFace.detectFace(temp_img_path, detector_backend = best_backend)
#                         pred_cnt+=1
#                     except ValueError:
# #                     except Exception as e:
# #                         print(e)
#                         continue
                    
#                     if pred_cnt>max_pred_cnt:
#                         break
                        
#                     # for every model, for every labeled img folder, 
#                     # avearge distance from labeled and detected img
#                     for j, model in enumerate(serengil_deep_face_models_list):
#                         # TODO: find a way to not print warnings or tqdms
                        
#                         # faster to pass build model for multiple calls
# #                         actual_model = DeepFace.build_model(model)
#                         df = stopPrint(DeepFace.find, 
#                                            img_path = temp_img_path, 
#                                            detector_backend = best_backend,
# #                                            model = actual_model,
#                                            model_name = model,
#                                            distance_metric = deep_face_metric,
#                                            enforce_detection = False,
#                                            db_path = sgdf_known_faces_dir)
                        
#                         df['identity'] = df['identity'].apply(change_img_name)
#                         # distance_col = "distance"
#                         distance_col = model+"_"+deep_face_metric
#                         df = df.groupby('identity', as_index=False)[distance_col].mean()
#                         if df.empty:
#                             y_pred_dict[model].append(0)
#                             continue
#                         optim_folder_name = df.sort_values('identity', ascending=deep_face_metric_ascending).iloc[0]['identity']
#                         name = folder_name_2_prof_name_dict[optim_folder_name]
#                         names_dict_list[model].append(name)
#                         frame_index_dict_list[model].append(int(i/fps))
#                         if name==true_name:
#                             y_pred_dict[model].append(1)
#                         else:
#                             y_pred_dict[model].append(0)

#                 # Hit 'q' on the keyboard to quit!
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break

#             # Release handle to the webcam
#             video_capture.release()
#             cv2.destroyAllWindows()
            
#             print(f"\n\nvedio: {vname}")
#             print(f"true_name: {true_name}")
#             y_pred_list_list = []
#             for j, model in enumerate(serengil_deep_face_models_list):
#                 y_pred = y_pred_dict[model]
#                 y_pred_list_list.append(y_pred)
#                 names_list = names_dict_list[model]
#                 frame_index_list = frame_index_dict_list[model]
#                 print(f"\nmodel: {model}")
#                 print(f"unique predicted value counts:")
#                 unique, counts = np.unique(np.array(names_list), return_counts=True)
#                 print(np.asarray((unique, counts)).T)
#                 print(f"accuracy:{np.array(y_pred).mean()} \
#                 \nnumber of predictions: {len(y_pred)}")
#                 results_dict = {"vname": vname,
#                             "nptel_channel_name": nptel_channel_name, 
#                             "playlistn": playlistn,
#                             "vedio_path": vedio_path,
#                                 "model": model,
#                             "true_name": true_name,
#                             "accuracy": np.array(y_pred).mean(),
#                             "number of predictions": len(y_pred),
#                             "time": tt()-st,
#                             "unique": unique,
#                             "counts": counts,
#                             "frame_index_list":frame_index_list,
#                             "names_list":names_list,
#                             }
#                 results_dict_list_dict[model].append(results_dict)
                
#             mean_y_pred_list = list(np.array(y_pred_list_list).mean(axis=1))
#             best_model_idx = mean_y_pred_list.index(max(mean_y_pred_list))
#             best_model = serengil_deep_face_models_list[best_model_idx]
#             best_model_list.append(best_model)
#             best_acc = mean_y_pred_list[best_model_idx]
#             print(f"\nbest model: {model} \n accuracy:{best_acc} \n time: {tt()-st}")
#             best_results_list_dict.append(results_dict_list_dict[best_model][-1])
            
#             # save results csv files 
#             for j, model in enumerate(serengil_deep_face_models_list):
#                 results_dict_list = results_dict_list_dict[model]
#                 results_dict_csv = pd.DataFrame(results_dict_list)
#                 results_dict_csv.to_csv(os.path.join(csv_files_dir, f"serengil_deep_face_results_{model}.csv"))
#             results_dict_csv = pd.DataFrame(best_results_list_dict)
#             results_dict_csv.to_csv(os.path.join(csv_files_dir, f"serengil_deep_face_results_best.csv"))

# print("unique best model contender counts:")
# unique, counts = np.unique(np.array(best_model_list), return_counts=True)
# print(np.asarray((unique, counts)).T)
# best_model = max(best_model_list, key=best_model_list.count)
# print(f"\nbest of the best model: {best_model}")

